In [2]:
#pip install earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.9/324.9 kB 1.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 602.2 kB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 854.7 kB/s eta 0:00:000:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 642.1 kB/s eta 0:00:000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 945.3 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 964.4 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 289.3 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 700.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 932.5 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 482.1 kB/s eta 0:00:000:00:010m
Note: you may need t

In [3]:
import ee

In [4]:
ee.Authenticate() 
ee.Initialize()

Enter verification code:  4/1AeaYSHDx0NEkHkG89LxZrLeT9kLXV4RS6eJRT5DS0ch4ihrtVcCbQ_xDiV4



Successfully saved authorization token.


In [20]:

def processYear(year):
    # Load the CDL dataset for the given year
    dataset = ee.ImageCollection('USDA/NASS/CDL')\
                .filter(ee.Filter.date(f'{year}-01-01', f'{year}-12-31'))\
                .first()
    crop_landcover = dataset.select('cropland')

    # Filter for North Dakota counties
    counties = ee.FeatureCollection('TIGER/2016/Counties')
    nd = counties.filter(ee.Filter.eq('STATEFP', '38'))
    
    # Identify corn areas in North Dakota
    corn = crop_landcover.eq(1).Or(crop_landcover.eq(5)).Or(crop_landcover.eq(6))
    masked_corn = crop_landcover.updateMask(corn).clipToCollection(nd)

    # Calculate NDVI for corn areas using MODIS data
    NDVI_dataset = ee.ImageCollection('MODIS/061/MOD13Q1')\
                    .filter(ee.Filter.date(f'{year}-05-01', f'{year}-10-01'))
    ndvi = NDVI_dataset.select('NDVI')
    mean_ndvi = ndvi.mean().rename('NDVI')
    cornNDVI = mean_ndvi.updateMask(masked_corn)
    
    # Calculate precipitation using GRIDMET data
    precipitation_dataset = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")\
                             .filter(ee.Filter.date(f'{year}-05-01', f'{year}-10-01'))\
                             .select('pr')
    mean_precipitation = precipitation_dataset.mean().rename('PA')

    # Load Sentinel-1 C-band SAR Image Collection for the given year, select VV polarization
    s1_dataset = ee.ImageCollection("COPERNICUS/S1_GRD")\
                   .filter(ee.Filter.eq('instrumentMode', 'IW'))\
                   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
                   .filter(ee.Filter.date(f'{year}-01-01', f'{year}-12-31'))\
                   .select('VV')
    mean_s1_vv = s1_dataset.mean().rename('SAR')

    # Load Radiometer Global Daily 9 km Soil Moisture AM
    smap_dataset = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/005")\
                    .filter(ee.Filter.date(f'{year}-05-01', f'{year}-10-01'))\
                    .select('soil_moisture_am')
    mean_soil_moisture = smap_dataset.mean().rename('SMS_AM')

    # Load MODIS Land Surface Temperature DAY
    lstDataset = ee.ImageCollection("MODIS/061/MOD11A1")\
                   .filter(ee.Filter.date(f'{year}-05-01', f'{year}-10-01'))
 
    lstmean_celsius = lstDataset.select('LST_Day_1km')\
                                .mean()\
                                .multiply(0.02)\
                                .subtract(273.15)\
                                .rename('LST_DAY')
                         
    # Load Radiometer Global Daily 9 km Soil Moisture PM

    smapDataset_pm = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/005")\
                       .filter(ee.Filter.date(f'{year}-05-01', f'{year}-10-01'))\
                       .select('soil_moisture_am') 
    meanSoilMoisture_pm = smapDataset_pm.mean().rename('SMS_PM')

                         
  
    # Load MODIS Land Surface Temperature NIGHT
    lstDataset_night = ee.ImageCollection("MODIS/061/MOD11A1")\
                         .filter(ee.Filter.date(f'{year}-05-01', f'{year}-10-01'))
 
    lstmean_celsius_night = lstDataset_night.select('LST_Night_1km')\
                                              .mean()\
                                              .multiply(0.02)\
                                              .subtract(273.15)\
                                              .rename('LST_NIGHT')
                         
    # Photosynthetically Active Radiation Daily 3-Hour 

    par_12 = ee.ImageCollection("MODIS/061/MCD18C2")\
               .filter(ee.Filter.date(f'{year}-05-01', f'{year}-10-01'))\
               .select('GMT_1200_PAR')
                        
    mean_par_12 = par_12.mean().rename('PAR'); # Calculate the Photosynthetically Active Radiation at 12

                         
    # Net Evapotranspiration  
    netevapo = ee.ImageCollection("MODIS/061/MOD16A2GF")\
                 .filter(ee.Filter.date(f'{year}-05-01', f'{year}-10-01'))\
                 .select('ET')
                    
    mean_netevapo = netevapo.mean().rename('ET')  # Calculate the mean Soil Moisture


    # Combine all layers
    combinedDataset = cornNDVI.addBands(mean_precipitation).addBands(mean_s1_vv).addBands(mean_soil_moisture).addBands(lstmean_celsius).addBands(meanSoilMoisture_pm).addBands(lstmean_celsius_night).addBands(mean_par_12).addBands(mean_netevapo)

    # Reduce regions and calculate mean values over the specified areas
    combined_mean = combinedDataset.reduceRegions(
        collection=nd,
        reducer=ee.Reducer.mean(),
        scale=30,
        tileScale=4,
    )

    # Define export parameters
    export_params = {
        'collection': combined_mean,
        'description': f'county_combined_{year}',
        'folder': 'GEE_Folder',
        'fileNamePrefix': f'Combined_Summary_By_County_{year}',
        'fileFormat': 'CSV',
        'selectors': ['NAME', 'GEOID', 'NDVI', 'PA', 'SAR', 'SMS_AM', 'LST_DAY', 'SMS_PM', 'LST_NIGHT', 'PAR', 'ET']
    }

    # Uncomment the line below to as I have got the data in my drive 
    #ee.batch.Export.table.toDrive(**export_params).start()

# Example of processing each year
for year in range(2023, 2024):
    processYear(year)
